In [2]:
# def decode(x1, x2, x3, x4):
#     return [0] * x1 + [1] * x2 + [2] * x3 + [3] * x4
#
# def okay(plan):
#     if len(plan) <= 21:
#         return False
#     if not plan:
#         return False
#     return plan[-1] == 3 and 1 in plan and 2 in plan
#
# TOTAL1 = 31
# alls = []
# for x1 in range(0, TOTAL1 + 1):
#     TOTAL2 = TOTAL1 - x1
#     for x2 in range(0, TOTAL2 + 1):
#         TOTAL3 = TOTAL2 - x2
#         for x3 in range(0, TOTAL3 + 1):
#             TOTAL4 = TOTAL3 - x3
#             for x4 in range(0, TOTAL4 + 1):
#                 plan = decode(x1, x2, x3, x4)
#                 if okay(plan):
#                     #plan = str(plan).replace('[', '').replace(']', '').replace(' ', '')
#                     alls.append(plan)
#
# import random
# random.shuffle(alls)

In [3]:
def decode(x1, x2, x3, x4):
    return [0] * x1 + [1] * x2 + [2] * x3 + [3] * x4

def encode(x1, x2, x3, x4):
    return x1, x2, x3, x4

def monotonic(it):
    if len(it) == 0:
        return True
    prev = it[0]
    for element in it[1:]:
        if prev > element:
            return False
        prev = element
    return True

def okay(plan):
    if len(plan) <= 3:
        return False
    if not plan:
        return False
    return plan[-1] == 3 and 1 in plan and 2 in plan

MAX_LENGTH = -1
TOTAL1 = 31
alls = set()
for x1 in range(0, TOTAL1 + 1):
    TOTAL2 = TOTAL1 - x1
    for x2 in range(0, TOTAL2 + 1):
        TOTAL3 = TOTAL2 - x2
        for x3 in range(0, TOTAL3 + 1):
            TOTAL4 = TOTAL3 - x3
            for x4 in range(0, TOTAL4 + 1):
                plan = decode(x1, x2, x3, x4)
                if okay(plan):
                    if len(plan) > MAX_LENGTH:
                        MAX_LENGTH = len(plan)
                    plan = encode(x1, x2, x3, x4)
                    alls.add(plan)

print('MAX_LENGTH =', MAX_LENGTH)
print('# PLANS (BEFORE INSERTION) =', len(alls))

alls = list(alls)
import random
random.shuffle(alls)

MAX_LENGTH = 31
# PLANS (BEFORE INSERTION) = 35959


In [4]:
def single_insertions(encoded):
    first = []
    for i in [0, 1, 2, 3]:
        first.extend([i] * encoded[i])
    yield tuple(first)
    if len(first) >= MAX_LENGTH:
        return
    for thing_to_insert in {0, 1, 2, 3}:
        for place_to_insert in {0, 1, 2, 3} - {thing_to_insert}:
            for insertion_index in range(0, encoded[place_to_insert]+1):
                rv = []
                for i in range(4):
                    if i == place_to_insert:
                        rv += [place_to_insert] * insertion_index + [thing_to_insert] + [place_to_insert] * (encoded[place_to_insert] - insertion_index)
                    else:
                        rv += [i] * encoded[i]
                if not monotonic(rv):
                    yield tuple(rv)

In [5]:
import tqdm.notebook as tqdm

MAX_LENGTH2 = -1
with_insert = set()
for a in tqdm.tqdm(alls):
    for r in single_insertions(a):
        with_insert.add(r)
        if len(r) > MAX_LENGTH2:
            MAX_LENGTH2 = len(r)

print('MAX_LENGTH2 =', MAX_LENGTH2)
print('# PLANS (AFTER INSERTION) =', len(with_insert))



  0%|          | 0/35959 [00:00<?, ?it/s]

MAX_LENGTH2 = 31
# PLANS (AFTER INSERTION) = 2275852


In [6]:
with open('plans_1_insert.csv', 'w') as file:
    for plan in tqdm.tqdm(with_insert):
        print(','.join([str(e) for e in plan]), end='\n', file=file)

  0%|          | 0/2275852 [00:00<?, ?it/s]

In [7]:
def decode2nd(x):
    x0, x1, x2, x3 = x
    return tuple(x0 + x1 + x2 + x3)

def insert_in_list(lst, thing_to_insert):
    for insertion_index in range(0, len(lst)+1):
        yield lst[:insertion_index] + [thing_to_insert] + lst[insertion_index:]

def second_insert(x):
    decoded = decode2nd(x)
    yield tuple(decoded)
    if len(decoded) >= MAX_LENGTH:
        return
    x = [list(e) for e in x] # copy
    for thing_to_insert in {0, 1, 2, 3}:
        for place_to_insert in {0, 1, 2, 3} - {thing_to_insert}:
            for rv in insert_in_list(x[place_to_insert], thing_to_insert):
                temp = []
                for i in range(4):
                    if i == place_to_insert:
                        temp.extend(rv)
                    else:
                        temp.extend(x[i])
                yield tuple(temp)

def double_insertions(encoded):
    first = []
    for i in [0, 1, 2, 3]:
        first.extend([i] * encoded[i])
    yield tuple(first)
    if len(first) >= MAX_LENGTH:
        return
    for thing_to_insert in {0, 1, 2, 3}:
        for place_to_insert in {0, 1, 2, 3} - {thing_to_insert}:
            for insertion_index in range(0, encoded[place_to_insert]+1):
                temp = []
                for i in range(4):
                    if i == place_to_insert:
                        temp.append([place_to_insert] * insertion_index + [thing_to_insert] + [place_to_insert] * (encoded[place_to_insert] - insertion_index))
                    else:
                        temp.append([i] * encoded[i])
                for inner in second_insert(temp):
                    yield inner

In [10]:
import tqdm.notebook as tqdm

MAX_LENGTH2 = -1
with_insert = set()
i = 0
with tqdm.tqdm(total=len(alls)) as pbar:
    for a in alls:
        for r in double_insertions(a):
            with_insert.add(r)
            if len(r) > MAX_LENGTH2:
                MAX_LENGTH2 = len(r)
        i += 1
        if i % 10 == 0 and i > 0:
            pbar.desc = '# with insert = ' + str(len(with_insert))
            pbar.update(10)
        if len(with_insert) > 12_000_000:
            with_insert = list(with_insert)
            random.shuffle(with_insert)
            with_insert = set(with_insert[:10_000_000])

print('MAX_LENGTH2 =', MAX_LENGTH2)
print('# PLANS (AFTER INSERTION) =', len(with_insert))



  0%|          | 0/35959 [00:00<?, ?it/s]

MAX_LENGTH2 = 31
# PLANS (AFTER INSERTION) = 10818655


In [11]:
with_insert = list(with_insert)
random.shuffle(with_insert)
with_insert = set(with_insert[:10_000_000])

print(len(with_insert))

10000000


In [12]:
with open('plans_2_insert_sample.csv', 'w') as file:
    for plan in tqdm.tqdm(with_insert):
        print(','.join([str(e) for e in plan]), end='\n', file=file)

  0%|          | 0/10000000 [00:01<?, ?it/s]

In [13]:
wi = list(with_insert)
wi[0]

(0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3)